# Assignment Part 1 Brief 

## Deadline: Friday, December 3, 2021 at 15:00 hrs

## Number of marks available for Part 1: 25

## Scope: Sessions 1 to 5

### Please READ the whole assignment first, before starting to work on it.

### How and what to submit

A. A **Jupyter Notebook** with the code in all the cells executed and outputs displayed.

B. Name your Notebook **COM4509-6509_Assignment_Part1_Username_XXXXXX.ipynb** where XXXXXX is your username such as such as abc18de.  

C. Upload a .zip file to Blackboard before the deadline that contains the Jupyter Notebook in B and any other files requested for the solution of Part 2 of the Assignment (Dr Lu will be in charge of releasing this part). 

D. **NO DATA UPLOAD**: Please do not upload the data files used in this Notebook. We have a copy already. 


### Assessment Criteria 

* Being able to use numpy and pandas to preprocess a dataset.

* Being able to use numpy to build a machine learning pipeline for supervised learning. 

* Being able to follow the steps involved in an end-to-end project in machine learning.

* Being able to use scikit-learn to design a machine learning model pipeline


### Late submissions

We follow Department's guidelines about late submissions, i.e., a deduction of 5% of the mark each working day the work is late after the deadline. NO late submission will be marked one week after the deadline because we will release a solution by then. Please read [this link](https://sites.google.com/sheffield.ac.uk/comughandbook/general-information/assessment/late-submission) if you are taking COM4509 or read [this link](https://sites.google.com/sheffield.ac.uk/compgtstudenthandbook/menu/assessment/late-submission) if you are taking COM6509. 

### Use of unfair means 

**"Any form of unfair means is treated as a serious academic offence and action may be taken under the Discipline Regulations."** (from the students Handbook). Please carefully read [this link](https://sites.google.com/sheffield.ac.uk/comughandbook/general-information/assessment/unfair-means) on what constitutes Unfair Means if not sure, for COM4509. If you are taking COM6509, please read [this link](https://sites.google.com/sheffield.ac.uk/compgtstudenthandbook/menu/referencing-unfair-means) if you are not sure what is Unfair means. If you still have questions, please ask your Personal tutor or the Lecturers. 

# A dataset of air quality

The dataset you will use in this assignment comes from a popular machine learning repository that hosts open source datasets for educational and research purposes, the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php). We are going to use regularised ridge regression and random forests for predicting air quality. The description of the dataset can be found [here](https://archive.ics.uci.edu/ml/datasets/Air+Quality).

In [2]:
import urllib.request
doq = "https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip"
pat_sav = "./AirQualityUCI.zip"
urllib.request.urlretrieve(doq, pat_sav)
#urllib.request.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip")

('./AirQualityUCI.zip', <http.client.HTTPMessage at 0x1b1f9b7c50>)

In [3]:
import zipfile
zip = zipfile.ZipFile('./AirQualityUCI.zip', 'r')
for name in zip.namelist():
    zip.extract(name, '.')

In [4]:
# The .csv version of the file has some typing issues, so we use the excel version
import pandas as pd 
air_quality_full = pd.read_excel('./AirQualityUCI.xlsx', usecols=range(2,15))

We can see some of the rows in the dataset 

In [5]:
air_quality_full.sample(5)

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
2675,0.7,951.50,-200,4.581919,742.0,49.0,892.00,57.0,1656.25,953.00,24.575,59.150000,1.801287
8726,4.3,1449.75,-200,20.927084,1322.0,876.5,489.75,189.5,1497.25,1755.50,3.300,61.625001,0.482899
6624,2.4,1005.75,-200,10.742267,1005.0,350.0,781.00,133.0,1071.25,1044.50,14.875,24.850000,0.417807
5095,1.3,1157.50,-200,8.574603,922.5,142.0,673.25,62.0,1519.25,1071.75,21.900,61.525001,1.596728
6725,1.7,-200.00,-200,-200.000000,-200.0,242.0,-200.00,96.0,-200.00,-200.00,-200.000,-200.000000,-200.000000


The target variable corresponds to the CO(GT) variable of the first column. The following columns correspond to the variables in the feature vectors, *e.g.*, PT08.S1(CO) is $x_1$ up until AH which is $x_D$. The original dataset also has a date and a time columns that we are not going to use in this assignment.

### Removing instances 

The dataset has missing values tagged with a -200 value. To simplify the design of the machine learning models in this assignment, we perform the following two operations to the dataset right from the beginning:

* we will remove the rows for which the target variable has missing values. We are doing supervised learning so we need all our data observations to have known target values.

* we will remove features with more than 20% of missing values. 

The code below performs both operations

In [6]:
# We first remove the rows for which there are missing values in the target feature
air_quality = air_quality_full.loc[air_quality_full.iloc[:, 0]!=-200, :]

In [7]:
# We now remove the columns (features) for which there are more that 20% of missing values
import numpy as np
ndata, ncols = np.shape(air_quality) # number of data observations and number of columns in the dataframe

pmissing = np.empty(ncols)         # An empty vector that will keep the percentage of missing values per feature
for i in range(ncols):
    pmissing[i] = (air_quality.iloc[:, i]==-200).sum()/ndata # Computes the percentage of missing values per column
air_quality = air_quality.loc[:, pmissing < 0.2]
print(ndata,ncols)

7674 13


### Splitting the dataset 

Before designing any machine learning model, we need to set aside the test data. We will use the remaining training data for fitting the model. *It is important to remember that the test data has to be set aside before preprocessing*. 

Any preprocessing that you do has to be done only on the training data and several key statistics need to be saved for the test stage.  Separating the dataset into training and test before any preprocessing has happened help us to recreate the real world scenario where we will deploy our system and for which the data will come without any preprocessing.

Furthermore, we are going to use *hold-out validation* for validating our predictive model so we need to further separate the training data into a training set and a validation set.

We split the dataset into a training set, a validation set and a test set. The training set will have 70% of the total observations, the validation set 15% and the test set, the remaining 15%. For making the random selections of the training and validation sets **make sure that you use a random seed that corresponds to the last five digits of your student UCard**. In the code below, I have used 55555 as an example of my random seed.

In [8]:
np.random.seed(16384)                 # Make sure you use the last five digits of your student UCard as your seed
index = np.random.permutation(ndata)  # We permute the indexes 
N = np.int64(np.round(0.70*ndata))    # We compute N, the number of training instances
Nval = np.int64(np.round(0.15*ndata)) # We compute Nval, the number of validation instances   
Ntest = ndata - N - Nval              # We compute Ntest, the number of test instances
data_training_unproc = air_quality.iloc[index[0:N], :].copy() # Select the training data
data_val_unproc = air_quality.iloc[index[N:N+Nval], :].copy() # Select the validation data
data_test_unproc = air_quality.iloc[index[N+Nval:ndata], :].copy() # Select the test data

The assigment is divided into two sections. In the **first section**, you will design a regularised ridge regression model trained with stochastic gradient descent. You will write all the code from scratch. You should not use any library that already implements any of the routines considered in this section, for example, scikit-learn. In the **second section**, you will design a random forests model and you are allowed to use scikit-learn in this section.

When writing your code, you will find out that there are operations that are repeated at least twice. We will assign marks for use of Python functions and for commenting your code. The marks will be assigned as:

* Did you include Python functions to solve the question and avoid repeating code? (**1 mark**)
* Did you comment your code to make it readable to others? (**1 mark**)

# 1. Using regularised ridge regression to predict air quality (10 marks)

**DO NOT USE scikit-learn or any other machine learning library for the questions on this section. You are meant to write Python code from scratch. You can use Pandas and Numpy. Using scikit-learn or any other machine learning library for the questions in this section will give ZERO marks. No excuse will be accepted.**

Regularisation is a technique commonly used in Machine Learning to prevent overfitting. It consists on adding terms to the objective function such that the optimisation procedure avoids solutions that just learn the training data. Popular techniques for regularisation in Supervised Learning include [Lasso Regression](https://en.wikipedia.org/wiki/Lasso_(statistics)), [Ridge Regression](https://en.wikipedia.org/wiki/Tikhonov_regularization) and the [Elastic Net](https://en.wikipedia.org/wiki/Elastic_net_regularization). 

In this part of the Assignment, you will be looking at Ridge Regression and implementing equations to optimise the objective function using the update rules for stochastic gradient descent. You will use those update rules for making predictions on the Air Quality dataset.

## 1.1 Ridge Regression

Let us start with a data set for training $\mathcal{D} = \{\mathbf{y}, \mathbf{X}\}$, where the vector $\mathbf{y}=[y_1, \cdots, y_N]^{\top}$ and $\mathbf{X}$ is the design matrix from Lab 4, this is, 

\begin{align*}
    \mathbf{X} = 
                \begin{bmatrix}
                        1 & x_{1,1} & \cdots & x_{1, D}\\
                        1 & x_{2,1} & \cdots & x_{2, D}\\
                   \vdots &  \vdots\\
                        1 & x_{N,1} & \cdots & x_{N, D}
                \end{bmatrix}
               = 
               \begin{bmatrix}
                      \mathbf{x}_1^{\top}\\
                       \mathbf{x}_2^{\top}\\
                          \vdots\\
                        \mathbf{x}_N^{\top}
                \end{bmatrix}.
\end{align*}

Our predictive model is going to be a linear model

$$ f(\mathbf{x}_i) = \mathbf{w}^{\top}\mathbf{x}_i,$$

where $\mathbf{w} = [w_0\; w_1\; \cdots \; w_D]^{\top}$.

The **objective function** we are going to use has the following form

$$ E(\mathbf{w}, \lambda) = \frac{1}{N}\sum_{n=1}^N (y_n - f(\mathbf{x}_n))^2 + \frac{\lambda}{2}\sum_{j=0}^D w_j^2,$$

where $\lambda>0$ is known as the *regularisation* parameter.

This objective function was studied in Lecture 4. 

The first term on the rhs is what we call the "fitting" term whereas the second term in the expression is the regularisation term. Given $\lambda$, the two terms in the expression have different purposes. The first term is looking for a value of $\mathbf{w}$ that leads the squared-errors to zero. While doing this, $\mathbf{w}$ can take any value and lead to a solution that it is only good for the training data but perhaps not for the test data. The second term is regularising the behavior of the first term by driving the $\mathbf{w}$ towards zero. By doing this, it restricts the possible set of values that $\mathbf{w}$ might take according to the first term. The value that we use for $\lambda$ will allow a compromise between a value of $\mathbf{w}$ that exactly fits the data (first term) or a value of $\mathbf{w}$ that does not grow too much (second term).

This type of regularisation has different names: ridge regression, Tikhonov regularisation or $\ell_2$ norm regularisation. 

## 1.2 Optimising the objective function with respect to $\mathbf{w}$

There are two ways we can optimise the objective function with respect to $\mathbf{w}$. The first one leads to a closed form expression for $\mathbf{w}$ and the second one using an iterative optimisation procedure that updates the value of $\mathbf{w}$ at each iteration by using the gradient of the objective function with respect to $\mathbf{w}$,
$$
\mathbf{w}_{\text{new}} = \mathbf{w}_{\text{old}} - \eta \frac{d E(\mathbf{w}, \lambda)}{d\mathbf{w}},
$$
where $\eta$ is the *learning rate* parameter and $\frac{d E(\mathbf{w}, \lambda)}{d\mathbf{w}}$ is the gradient of the objective function.

It can be shown (this is a question in the Exercise Sheet 4) that a closed-form expression for the optimal $\mathbf{w}_*$ is given as

\begin{align*}            
            \mathbf{w}_*& = \left(\mathbf{X}^{\top}\mathbf{X} + \frac{\lambda N}   
                                     {2}\mathbf{I}\right)^{-1}\mathbf{X}^{\top}\mathbf{y}.
\end{align*}

Alternatively, we can find an update equation for $\mathbf{w}_{\text{new}}$ using gradient descent leading to:

\begin{align*}
   \mathbf{w}_{\text{new}} & = \mathbf{w}_{\text{old}} - \eta \frac{d E(\mathbf{w}, \lambda)}
                              {d\mathbf{w}},\\
                           & = \mathbf{w}_{\text{old}} +  \frac{2\eta}{N}\sum_{n=1}^N   
                               \left(y_n - \mathbf{x}_n^{\top}\mathbf{w}_{\text{old}}\right)\mathbf{x}_n  
                       - \eta\lambda\mathbf{w}_{\text{old}}\\
                           & = (1 - \eta\lambda)\mathbf{w}_{\text{old}} + \frac{2\eta}
                               {N}\sum_{n=1}^N   
                               \left(y_n - \mathbf{x}_n^{\top}\mathbf{w}_{\text{old}}\right)\mathbf{x}_n
\end{align*}

## 1.3 Preprocessing the data

As mentioned before, the dataset has missing values tagged with a -200 value. Before doing any work with the training data, we want to make sure that we deal properly with the missing values. Furthermore, once we have dealt with the missing values, we want to standardise the training data. 

### Question 1.a: Missing values and standardisation (2 marks)

* For all the other features with missing values, use the mean value of the non-missing values for that feature to perform imputation. Save these mean values, you will need them when performing the validation stage (**1 mark**).

* Once you have imputed the missing data, we need to standardise the input vectors. Standardise the training data by substracting the mean value for each feature and dividing the result by the standard deviation of each feature. Keep the mean values and standard deviations, you will need them at validation time (**1 mark**).

#### Question 1.a Answer

In [33]:
# Answer 1.a
def impute(dataset,data_mean_cols):
    dataset = dataset.replace(-200, np.nan)
    for column in list(dataset.columns):
        dataset[column].fillna(data_mean_cols[column],inplace = True)
    return dataset
    
def mean_std(dataset):
    data_mean_cols = dataset.mean()
    data_std_cols = dataset.std()
    return(data_mean_cols,data_std_cols)
    
def standardise(dataset,data_mean_cols,data_std_cols):
    for column in list(dataset.columns):
        dataset[column] = (dataset[column] - data_mean_cols[column])/data_std_cols[column]
    return dataset

data_train_q1 = data_training_unproc.copy()
train_mean_q1,train_std_q1 = mean_std(data_train_q1)
print(train_mean_q1,train_std_q1)
imp_train_q1 = impute(data_train_q1,train_mean_q1)
sta_train_q1 = standardise(imp_train_q1,train_mean_q1,train_std_q1)
sta_train_q1.isnull().any()

CO(GT)              2.163980
PT08.S1(CO)      1058.358107
C6H6(GT)            1.662090
PT08.S2(NMHC)     902.156475
NOx(GT)           231.606069
PT08.S3(NOx)      781.549330
NO2(GT)            97.699348
PT08.S4(NO2)     1379.118733
PT08.S5(O3)       995.838623
T                   8.823145
RH                 38.738134
AH                 -7.314541
dtype: float64 CO(GT)             1.460052
PT08.S1(CO)      337.973561
C6H6(GT)          42.516619
PT08.S2(NMHC)    347.585264
NOx(GT)          233.419210
PT08.S3(NOx)     321.847535
NO2(GT)           85.759938
PT08.S4(NO2)     475.456278
PT08.S5(O3)      469.179034
T                 44.226487
RH                52.456259
AH                40.011334
dtype: float64


CO(GT)           False
PT08.S1(CO)      False
C6H6(GT)         False
PT08.S2(NMHC)    False
NOx(GT)          False
PT08.S3(NOx)     False
NO2(GT)          False
PT08.S4(NO2)     False
PT08.S5(O3)      False
T                False
RH               False
AH               False
dtype: bool

## 1.4 Training and validation stages

We have now curated our training data by removing data observations and features with a large amount of missing values. We have also normalised the feature vectors. We are now in a good position to work on developing the prediction model and validating it. We will use gradient descent for iterative optimisation. 

We first organise the dataframe into the vector of targets $\mathbf{y}$, call it `yTrain`, and the design matrix $\mathbf{X}$, call it `XTrain`.

In [35]:
# Write your code here
ndata_q1,ncols_q1=np.shape(sta_train_q1)

xTrain = sta_train_q1.drop('CO(GT)', axis=1)
XTrain = np.hstack([np.ones((ndata_q1,1)), xTrain])     #design matrix X
yTrain = sta_train_q1['CO(GT)'].values


# Question 1.b: finding the optimal $\mathbf{w}$ with stochastic gradient descent (3 marks)

Use gradient descent to iteratively compute the value of $\mathbf{w}_{\text{new}}$. Instead of using all the training set in `XTrain` and `yTrain` to compute the gradient, use a subset of $S$ instances in `XTrain` and `yTrain`. This is sometimes called *minibatch gradient descent* where $S$ is the size of the minibacth. When using gradient descent with minibatches, you need to find the best values for three parameters: $\eta$, the learning rate, $S$, the number of datapoints in the minibatch and $\gamma$, the regularisation parameter.

* In this question we will use the validation data. So before proceeding to the next steps, make sure that you:  replace the missing values on each feature variables with the mean value you computed with the training data; standardise the validation data using the means and standard deviations computed from the training data (**1 mark**).
    
* Create a grid of values for the parameters $\gamma$ and $\eta$ using `np.logspace` and a grid of values for $S$ using `np.linspace`. Because you need to find three parameters, start with five values for each parameter in the grid and see if you can increase it. Make sure you understand what is the meaning of `np.logspace` and `np.linspace`. Notice that you can use negative values for `start` in `np.logspace` (**1 mark**).

* For each value that you have of $\gamma$, $\eta$ and $S$ from the previous step, use the training set to compute $\mathbf{w}$ using minibatch gradient descent and then measure the RMSE over the validation data. For the validation data, make sure you preprocess it before applying the prediction model over it. For the minibatch gradient descent choose to stop the iterative procedure after $200$ iterations (**1 mark**).

* Choose the values of $\gamma$, $\eta$ and $S$ that lead to the lowest RMSE and save them. You will use them at the test stage.

#### Question 1.b Answer

In [121]:
# Write your code here
import random

data_val_q1 = data_val_unproc.copy()
imp_val_q1 = impute(data_val_q1,train_mean_q1)
sta_val_q1 = standardise(imp_val_q1,train_mean_q1,train_std_q1)
ndata_val,ncols_val = np.shape(sta_val_q1)
xVal_q1 = sta_val_q1.drop('CO(GT)', axis=1)
XVal_q1 = np.hstack((np.ones([ndata_val,1]), xVal_q1))
yVal_q1 = sta_val_q1['CO(GT)'].values



def data_iter(batch_size, x, y, ndata):      # seperate data into minibatch
    indices = list(range(ndata))                      
    np.random.shuffle(indices)                                        # in no particular order
    for i in range(0, ndata, batch_size):
        batch_indices = np.array(indices[i:min(i + batch_size, ndata)])
        yield x[batch_indices,:], y[batch_indices]


def model_prediction_lr(x,w):      #linear regression 
    return np.dot(x, w)

def loss_function_rr(yTrue,yPred,w,gamma,batch_size):
    return ((yTrue - yPred)**2).sum()/batch_size + gamma/2 * (w**2).sum()    #objective function

def update_w(x,w,yTrue,yPred,lr,gamma,batch_size):    
    return (1 - lr*gamma) * w + 2*lr/batch_size * (np.dot((yTrue-yPred),x)).sum()  #iterative w


def mini_batch_rr(x,yTrue,w,gamma,lr,batch_size,ndata):       #
    Max_Niter = 200
    for Niter in range(Max_Niter):
        for X, y in data_iter(batch_size, x, yTrue, ndata):
            y_approx = model_prediction_lr(X,w)
            my_loss = loss_function_rr(y,y_approx,w,gamma,batch_size)
            w = update_w(X,w,y,y_approx,lr,gamma,batch_size)
        #if Niter%20==0 or Niter == Max_Niter-1:
            #print(f'Iteration = {Niter+1}, Loss = {my_loss:.8f}')
    return w

def rmse(yTrue,yPred):
    error_mod = np.sqrt(((yTrue - yPred)**2).mean())
    return error_mod

re_para = np.logspace(-3,0,5)
lr = np.logspace(-3,-2,5)
batch_size = np.linspace(10,300,5,dtype = 'int')

val_rmse = np.empty([5,5,5])
for i in range(len(re_para)):
    for j in range(len(lr)):
        for k in range(len(batch_size)):
            w = np.random.normal(0, 0.01, np.shape(XTrain)[1])
            train_data_w = mini_batch_rr(XTrain,yTrain,w,re_para[i],lr[j],batch_size[k],np.shape(XTrain)[0])
            pred_yval = model_prediction_lr(XVal_q1,train_data_w)
            val_rmse[i,j,k] = np.sqrt(((yVal_q1 - pred_yval)**2).mean())
val_rmse

array([[[0.70820803, 0.71661143, 0.71118025, 0.69638511, 0.71386338],
        [0.71034663, 0.7034845 , 0.70947059, 0.70899389, 0.71658201],
        [0.73169051, 0.71467222, 0.71737064, 0.70777195, 0.71472618],
        [0.70229513, 0.70764879, 0.72202477, 0.7059568 , 0.70243157],
        [0.70555307, 0.71401808, 0.70878998, 0.70547637, 0.70840749]],

       [[0.69861518, 0.70874049, 0.70394823, 0.69525647, 0.7038457 ],
        [0.71217585, 0.70682549, 0.71155103, 0.72499073, 0.71584018],
        [0.7080035 , 0.71325935, 0.70166094, 0.71981017, 0.69729441],
        [0.70551579, 0.70951103, 0.71782202, 0.70319857, 0.69824343],
        [0.7067321 , 0.7106022 , 0.70486074, 0.71098177, 0.69729482]],

       [[0.70625163, 0.70729765, 0.71004065, 0.70619084, 0.69446013],
        [0.70714211, 0.71532281, 0.70541373, 0.70271558, 0.69148237],
        [0.7079539 , 0.70984824, 0.70364675, 0.7101572 , 0.71583909],
        [0.70886824, 0.70535759, 0.70421522, 0.70518671, 0.71477108],
        [0.70574

In [122]:
a,b,c = np.where(val_rmse==np.min(val_rmse))
op_r = re_para[a]
op_l = lr[b]
op_b = batch_size[c]
print('regression parameter:',op_r ,' learning rate:',op_l,' batch_size:',op_b)


regression parameter: [0.03162278]  learning rate: [0.00177828]  batch_size: [300]


## 1.5 Test stage 

We now know which one is the best model, according to the validation data. We will now put together the training data and the validation data, perform the preprocessing as we did before, this is, treat the missing values and standardise the inputs. We will train the model again using the minibatch stochastic gradient descent and finally compute the RMSE over the test data.


### Question 1.c: combine the original training and original validation data and perform the preprocessing again to this new data (2 marks)

Put together the original training and validation dataset and perform the same preprocessing steps than before, these are: 

* for each feature, impute the missing values with the mean values of the non-missing values (**1 mark**) 

* stardardise the new training set (**1 mark**).

#### Question 1.c Answer

In [123]:
# Write your code here
data_train_q1c = data_training_unproc.copy()
data_val_q1c = data_val_unproc.copy()
data_test_q1 = data_test_unproc.copy()
new_train_q1 = np.vstack((data_train_q1c,data_train_q1c))
new_train_q1 = pd.DataFrame(new_train_q1)
new_train_q1.columns = data_train_q1c.columns

new_train_mean_q1,new_train_std_q1 = mean_std(new_train_q1)
imp_new_train_q1 = impute(new_train_q1,new_train_mean_q1)
sta_new_train_q1 = standardise(imp_new_train_q1,new_train_mean_q1,new_train_std_q1)


### Question 1.d: Preprocess the test data, train the model and predict over the test data (3 marks)

Preprocess the test data and train a new model using the new training set. Finally, report the RMSE over the test set:

* Preprocess the test data by imputing the missing data and standardising it (**1 mark**). 

* Use the best values of $\gamma$, $\eta$ and $S$ found in the validation set and train a new regularised linear model with stochastic gradient descent (**1 mark**).

* Report the RMSE over the test data (**1 mark**).

#### Question 1.d Answer

In [124]:
# Write your code here
imp_test_q1 = impute(data_test_q1,new_train_mean_q1)
sta_test_q1 = standardise(imp_test_q1,train_mean_q1,train_std_q1)

n_xTrain = sta_new_train_q1.drop('CO(GT)', axis=1)
n_XTrain = np.hstack([np.ones((np.shape(n_xTrain)[0],1)), n_xTrain])     #design matrix X
n_yTrain = sta_new_train_q1['CO(GT)'].values

n_xTest = sta_test_q1.drop('CO(GT)', axis=1)
n_XTest = np.hstack([np.ones((np.shape(n_xTest)[0],1)), n_xTest])     #design matrix X
n_yTest = sta_test_q1['CO(GT)'].values

n_w = np.random.normal(0, 0.01, np.shape(sta_train_q1)[1])  
new_train_data_w = mini_batch_rr(n_XTrain,n_yTrain,n_w,op_r[0],op_l[0],op_b[0],np.shape(n_XTrain)[0])

pred_ytest = model_prediction_lr(n_XTest,new_train_data_w)
test_rmse = np.sqrt(((n_yTest - pred_ytest)**2).mean())
test_rmse

0.7266567303867545

# 2. Random forests (13 marks)

**USE scikit-learn for the questions on this section.**

In section 1, you used a regularised ridge regression model trained with SGD to create a linear predictive model. In this part of the assignment, you will use **scikit-learn** to train a random forest for regression over the air quality dataset.

## 2.1 Preprocessing the data

As mentioned before, the dataset has missing values tagged with a -200 value. Before doing any work with the training data, we want to make sure that we deal properly with the missing values. Furthermore, once we have dealt with the missing values, we want to standardise the training data. 

### Question 2.a: Pipeline for missing values and standardisation (3 marks)

* Employ the `SimpleImputer` method in Scikit-learn to impute the missing values in each column using the mean value of the non-missing values, instead (**1 mark**).

* Standardise the data by substracting the mean value for each feature and dividing the result by the standard deviation of each feature. Employ the `StandardScaler` method (**1 mark**).

* Create a `Pipeline` that you can use to preprocess the data by filling missing values and then standardising the features (**1 mark**).

#### Question 2.a Answer

In [9]:
# Write your code here
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import GridSearchCV 
from sklearn.ensemble import RandomForestRegressor

data_train_q2 = data_training_unproc.copy()        #copy the dataset to use in this session
data_val_q2 = data_val_unproc.copy() 
data_test_q2 = data_test_unproc.copy()

pipe = Pipeline([('imm',SimpleImputer(missing_values=-200, strategy='mean')),('scaler', StandardScaler())])


### Question 2.b: Use the Pipeline to fit the training data and transform the validation data (2 marks)

In the previous question, you created a `Pipeline` for applying a `SimpleImputer` and a `StandardScaler`. Use the Pipeline to fit the training data (**1 mark**) and transform the validation data (**1 mark**).

#### Question 2.b Answer

In [27]:
# Write your code here
pipe.fit_transform(data_train_q2)    #fit transformed data_train
std_data_val_q2 = pipe.transform(data_val_q2)

yTrain_q2 = std_data_train_q2[:,0]
xTrain_q2 = std_data_train_q2[:,1:]
yVal_q2 = std_data_val_q2[:,0]
xVal_q2 = std_data_val_q2[:,1:]

array([[-1.13977771, -1.37904927, -1.12692832, ...,  0.12918745,
         1.29541269,  1.41359342],
       [ 0.50415231,  1.1786602 ,  0.6527489 , ...,  1.31467433,
        -1.94011239, -0.9211831 ],
       [-0.24931562, -0.87403869, -0.42823815, ..., -0.39610088,
        -1.10062481, -1.27827758],
       ...,
       [ 1.53160857,  1.57053908,  1.240556  , ...,  0.14353965,
         0.70223313,  0.8761421 ],
       [-0.18081853,  0.37624154,  0.36340557, ...,  0.90133268,
         0.57835036,  2.36057773],
       [ 0.36715814,  0.81360636,  0.33409286, ..., -0.17794827,
        -0.77561484, -0.85696417]])

## 2.2 Random forest to predict air quality 

We now use random forests to predict air quality. Remember that the tree ensemble in random forests is built by training individual regression trees on different subsets of the training data and using a subset of the available features. For regression, the prediction is the average of the individual predictions of each tree. Some of the parameters required in the Random Forest implementation in Scikit-learn include:

Some of the additional parameters required in the Random Forest implementation in Scikit-learn include

> **n_estimators** the total number of trees to train<p>
**max_features** number of features to use as candidates for splitting at each tree node. <p>
    **boostrap**: Whether bootstrap samples are used when building trees. If False, the whole dataset is used to build each tree.<p>
   **max_samples**: If bootstrap is True, the number of samples to draw from X to train each base estimator.

### Question 2.c: train a random forest (4 marks)

In this question, you will train a random forest for predicting over the validation data. Use cross-validation over the validation data to select the best set of paramaters for the random forest regressor. Parameters to include in your exploration are **n_estimators**, **max_features** and **max_samples**. Use `np.linspace` or `np.logspace` to define ranges of values to explore for each parameter and create a grid to be assessed over the validation data. Make sure you use the same validation data that was given to you.

* Use `PredefinedSplit` to tell the cross validator which instances to use for training and which ones for validation (**1 mark**).

* Create a grid of values to explore that include a range of at least five values for each parameter **n_estimators**, **max_features** and **max_samples** (**1 mark**). 

* Train a random forest for regression model that uses the grid of parameters you created before. Use `GridSearchCV` to find the best set of parameters by performing cross-validation over the predefined split. (**1 mark**).

* Print the best values in your grid for **n_estimators**, **max_features** and **max_samples** (**1 mark**).

#### Question 2.c Answer

In [31]:
# Write your code here
data_tra_val_x = np.vstack((xTrain_q2, xVal_q2))
data_tra_val_y = np.hstack((yTrain_q2, yVal_q2))

test_fold = np.zeros((np.shape(data_tra_val_x)[0], 1))
test_fold[0:np.shape(xTrain_q2)[0]] = -1
ps = PredefinedSplit(test_fold)

n_estimators_opts = np.linspace(1,300,5,dtype='int')
max_features_opts = np.logspace(-2,0,5)
max_samples_opts = np.logspace(-2,-1,6)
param_grid = dict(n_estimators = n_estimators_opts,max_features = max_features_opts,max_samples = max_samples_opts)
grid_reg_rfr = GridSearchCV(RandomForestRegressor(), param_grid=param_grid, cv=ps, scoring='neg_mean_squared_error')
grid_reg_rfr.fit(data_tra_val_x, data_tra_val_y)
print(grid_reg_rfr.best_params_)


{'max_features': 0.01, 'max_samples': 0.039810717055349734, 'n_estimators': 75}


In [32]:
rfr = RandomForestRegressor(random_state=42,n_estimators=grid_reg_rfr.best_params_['n_estimators'],max_samples=grid_reg_rfr.best_params_['max_samples'],max_features = grid_reg_rfr.best_params_['max_features'])
rfr.fit(xTrain_q2,yTrain_q2)
val_predictions = rfr.predict(xVal_q2)

from sklearn.metrics import mean_squared_error
error_mod = np.sqrt(mean_squared_error(yVal_q2, val_predictions))
print('The RMSE on the validation data is :',error_mod)


The RMSE on the validation data is : 0.4573684816318046


In [103]:
cv = pd.DataFrame(grid_reg_rfr.cv_results_)
cv

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_features,param_max_samples,param_n_estimators,params,split0_test_score,mean_test_score,std_test_score,rank_test_score
0,0.005367,0.0,0.002363,0.0,0.1,0.1,1,"{'max_features': 0.1, 'max_samples': 0.1, 'n_e...",-0.197868,-0.197868,0.0,242
1,0.206678,0.0,0.068332,0.0,0.1,0.1,75,"{'max_features': 0.1, 'max_samples': 0.1, 'n_e...",-0.085087,-0.085087,0.0,207
2,0.353613,0.0,0.153221,0.0,0.1,0.1,150,"{'max_features': 0.1, 'max_samples': 0.1, 'n_e...",-0.083228,-0.083228,0.0,201
3,0.606857,0.0,0.233789,0.0,0.1,0.1,225,"{'max_features': 0.1, 'max_samples': 0.1, 'n_e...",-0.083409,-0.083409,0.0,203
4,1.088862,0.0,0.193985,0.0,0.1,0.1,300,"{'max_features': 0.1, 'max_samples': 0.1, 'n_e...",-0.083769,-0.083769,0.0,204
...,...,...,...,...,...,...,...,...,...,...,...,...
245,0.033345,0.0,0.001633,0.0,1.0,1.0,1,"{'max_features': 1.0, 'max_samples': 1.0, 'n_e...",-0.063846,-0.063846,0.0,142
246,2.519788,0.0,0.065840,0.0,1.0,1.0,75,"{'max_features': 1.0, 'max_samples': 1.0, 'n_e...",-0.012646,-0.012646,0.0,19
247,5.007917,0.0,0.135003,0.0,1.0,1.0,150,"{'max_features': 1.0, 'max_samples': 1.0, 'n_e...",-0.012412,-0.012412,0.0,18
248,7.557071,0.0,0.199511,0.0,1.0,1.0,225,"{'max_features': 1.0, 'max_samples': 1.0, 'n_e...",-0.012313,-0.012313,0.0,15


### Question 2.d: train a new model over the whole training data and report the prediction over the test set (4 marks)


Now that we have identified the best paramaters of the regression model, we use these parameters to train a new model over the whole training data (`data_training` plus `data_val`). We apply this model to the test set and report the performance.

* Create a new preprocessing pipeline for taking care of the missing values and standardisation over the whole training data (**1 mark**).

* Apply the created preprocessing pipeline to the test data (**1 mark**).

* Fit a random forest regression model to the training data using the best parameters found in Question 2.c (**1 mark**).

* Compute the RMSE over the test data and report the result (**1 mark**).

#### Question 2.d Answer

In [22]:
# Write your code here
data_new_train_q2 = np.vstack((data_train_q2, data_val_q2))
data_new_train_q2 = pd.DataFrame(data_new_train_q2)
data_new_train_q2.columns = data_train_q2.columns

std_new_train_q2 = pipe.fit_transform(data_new_train_q2)
std_test_q2 = pipe.transform(data_test_q2)
new_yTrain_q2 = std_new_train_q2[:,0]
new_xTrain_q2 = std_new_train_q2[:,1:]
yTest_q2 = std_test_q2[:,0]
xTest_q2 = std_test_q2[:,1:]

rfr.fit(new_xTrain_q2,new_yTrain_q2)
test_predictions = rfr.predict(xTest_q2)

error_mod = np.sqrt(mean_squared_error(yTest_q2, test_predictions))
print('The RMSE on the validation data is :',error_mod)


The RMSE on the validation data is : 0.3187330116513184
